In [1]:
# GA ALGORITHM

class Chromosome:
    def __init__(self, *args):
        self.radius = 1
        if len(args) < 5:
            self.epsilon = args[3]
            self.initSensorNum = args[2]
            self.mode = args[0]
            self.space = args[1]
            self.placeHolders = []
            self.fitness = -1
            self.SensorPlaceHolderSetup()
            self.SensorConfigurationSetup()
              
        elif len(args) == 5:
            self.epsilon = args[3]
            self.placeHolders = []
            self.fitness = -1
            self.grid = args[0]
            self.mode = args[1]
            self.space = args[2]
            self.SensorPlaceHolderSetup()

        
                
    def frange(self, start, stop, step):
        steps = []
        while start <= stop:
            steps.append(start)
            start +=step
            
        return steps

    def SensorPlaceHolderSetup(self):
        Xs = self.frange(0, self.space[0], self.epsilon)
        Ys = self.frange(0, self.space[1], self.epsilon)
        
        import matplotlib.pyplot as plt
        

        for x in Xs:
          for y in Ys:
            self.placeHolders.append([x, y])

        # plt.scatter(np.array(self.placeHolders).transpose()[0], np.array(self.placeHolders).transpose()[1])
        # plt.show()

    def SensorConfigurationSetup(self):
        Xs = self.frange(0, self.space[0], self.epsilon)
        Ys = self.frange(0, self.space[1], self.epsilon)
        self.grid = np.zeros(len(Xs) * len(Ys)).tolist()

        i = 0
        while i < self.initSensorNum:
            cell = random.randrange(len(self.grid))
            if self.grid[cell] == 0:
                self.grid[cell] = 1
                i += 1

    def makeBoundaries(self, minValue, maxValue):
        m = self.epsilon * np.ceil(float(1/self.epsilon) * minValue)
        M = self.epsilon * np.floor(float(1/self.epsilon) * maxValue)
        return m, M

    def GetSensorConfiguration(self):
        from collections import Counter
        sensorLocations, sensorTypes = self.GetSensorLocations()

        summaryDict = Counter(sensorTypes)

        # TODO: DIFFERENT SENSOR TYPE DEFINITIONS SHOULD BE ADDED HERE:
        configurationSummary = []
        for key in summaryDict:
            if (key == 1):
                configurationSummary.append(['motion sensors', summaryDict[key]])

            elif (key == 2):
                configurationSummary.append(['beacon sensors', summaryDict[key]])

        configurationDetails = []
        for index, loc in enumerate(sensorLocations):
            if (sensorTypes[index] == 1):
                configurationDetails.append(tuple([loc, 'kitchen', 'motion sensors']))

            elif (sensorTypes[index] == 2):
                configurationDetails.append(tuple([loc, 'kitchen', 'beacon sensors']))

        return [[configurationSummary, [tuple(configurationDetails)]], self.radius]


    def GetSensorLocations(self):
        sensorLocations = []
        sensorTypes = []
        for index, sensorIndicator in enumerate(self.grid):
            if (sensorIndicator > 0):
                sensorLocations.append(self.placeHolders[index])
                sensorTypes.append(sensorIndicator)

        return sensorLocations, sensorTypes


class GA:
    chromosomes = []
    def __init__(self, population, initializationMethod, Data_path, epsilon, initSensorNum, maxSensorNum, radius, mutation_rate, crossover, survival_rate, reproduction_rate):
        self.population = population
        self.mode = initializationMethod
        self.Data_path = Data_path
        self.epsilon = epsilon
        self.initSensorNum = initSensorNum
        self.maxSensorNum = maxSensorNum
        self.radius = radius
        self.mutation_rate = mutation_rate
        self.crossover = crossover
        self.survival_rate = survival_rate
        self.reproduction_rate = reproduction_rate
        
        self.sensor_distribution, self.types, self.space, self.rooms, self.agentTraces = self.ModelsInitializations()
        
        for i in range(population):
            self.chromosomes.append(Chromosome(self.mode, self.space, self.initSensorNum, self.epsilon))

    def Mutation(self, chromosome):
        for i in range(len(chromosome.grid)):
          if (random.random() < mutation_rate):
              if (chromosome.grid[i] == 0):
                  chromosome.grid[i] = 1
              else:
                  chromosome.grid[i] = 0

        return chromosome            

    def GetNextGeneration(self):
        import copy
        self.newGeneration = []

        last_one = True
        if int(np.ceil( (1 - self.survival_rate) *  (self.population / 2))) % 2 == 0:
            last_one = False

        self.chromosomes.sort(key=lambda x: x.fitness, reverse=True)

        for i in range(int(np.floor((1 - self.survival_rate) *  (self.population / 2)))):
            valid_child = False
            while not valid_child:
                coin1 = random.randrange(0, len(self.chromosomes) * self.reproduction_rate)
                coin2 = random.randrange(0, len(self.chromosomes) * self.reproduction_rate)

                # coin1 = random.randrange(int(np.sum([c.fitness + 1 for c in self.chromosomes])))
                # coin2 = random.randrange(int(np.sum([c.fitness + 1 for c in self.chromosomes])))

                # for c in self.chromosomes:
                #     coin1 -= c.fitness + 1
                #     if coin1 <= 0:
                #         p1 = copy.deepcopy(c)
                #         break

                # for c in self.chromosomes:
                #     coin2 -= c.fitness + 1
                #     if coin2 <= 0:
                #         p2 = copy.deepcopy(c)
                #         break

                p1 = copy.deepcopy(self.chromosomes[coin1])
                p2 = copy.deepcopy(self.chromosomes[coin2])

                p1.grid, p2.grid = self.Crossover(p1.grid, p2.grid)

                child1 = Chromosome(p1.grid, p1.mode, p1.space, self.epsilon, None)
                child2 = Chromosome(p2.grid, p2.mode, p2.space, self.epsilon, None)

                if sum(child1.grid) <= self.maxSensorNum or sum(child2.grid) <= self.maxSensorNum:
                    valid_child = True

            self.newGeneration.append(self.Mutation(child1))
            self.newGeneration.append(self.Mutation(child2))

        if last_one == True:
            self.newGeneration.append(self.chromosomes[int(np.floor(self.population / 2))])

        self.chromosomes = self.chromosomes[0: int(self.survival_rate * len(self.chromosomes))]

        for ng in self.newGeneration:
            self.chromosomes.append(ng)

    def Selection(self):
        self.chromosomes.sort(key=lambda x: x.fitness, reverse=True)
        self.chromosomes = self.chromosomes[0:self.population]

    def Crossover(self, l, q):
        l = list(l)
        q = list(q)
              
        f1 = random.randint(0, len(l)-1)
        f2 = random.randint(0, len(l)-1)
        while f1 == f2:
            f1 = random.randint(0, len(l)-1)
            f2 = random.randint(0, len(l)-1)

        if f1 > f2:
            tmp = f1
            f1 = f2
            f2 = tmp

        
        # interchanging the genes
        for i in range(f1, f2):
            l[i], q[i] = q[i], l[i]
        
        return l, q

    def ModelsInitializations(self):
        #----- Space and agent models -----: 
        simworldname = self.Data_path + '/Configuration Files/simulationWorld2.xml'
        agentTraces = []
        directory = os.fsencode(self.Data_path + 'Agent Trace Files/')
            
        for file in os.listdir(directory):
            filename = os.fsdecode(file)
            if filename.endswith(".csv"): 
                agentTraces.append(self.Data_path + 'Agent Trace Files/' + filename)

        # Parsing the space model: 
        space, rooms = pf.ParseWorld(simworldname)

        xs = []
        for i in space:
          for j in i:
            xs.append(j)
        A = list(set(xs))
        A.sort()
        space = [A[-1], A[-2]]

        # User parameters 
        types, sensor_distribution = pf.GetUsersParameters()

        roomsList = []
        for room in sensor_distribution:
            roomsList.append(room)
              
        return sensor_distribution, types, space, rooms, agentTraces

    def RunFitnessFunction(self, simulateMotionSensors, simulateEstimotes, Plotting, iteration):       
        for index, chromosome in enumerate(self.chromosomes):
            if (chromosome.fitness == -1):
                files = []

                # sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
                # Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'
                
                sys.path.append('../../Codes/Sensor Simulator/')
                Data_path = '../../Codes/Sensor Simulator/'

                all_sensors = set([])

                for agentTrace in self.agentTraces:
                    filecoding = ' '#"_" + str(iteration) + "_c" + str(index + 1) + '(' + self.mode + ')'
                    df_ = sim_sis.RunSimulator(self.space, self.rooms, agentTrace, chromosome.GetSensorConfiguration() , simulateMotionSensors, simulateEstimotes, Plotting, filecoding , self.Data_path)
                    dataFile, sensors = self.PreProcessor(df_)
                    all_sensors.update(sensors)
                    #self.D = dataFile
                    files.append(dataFile)
                    
                

                # sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome')
                sys.path.append('../../Codes/CASAS/AL-Smarthome')
                
                # %cd 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome'

                import al
                import imp
                imp.reload(al)
                all_sensors = list(all_sensors)
                # chromosome.fitness = ((al.leave_one_out(files, sensors)[0]) - (np.sum(chromosome.grid) / len(chromosome.placeHolders))) * 100

                chromosome.fitness = (al.leave_one_out(files, all_sensors)[0] - (sum(chromosome.grid) / 100)) * 100 # - (len(chromosome.placeHolders)**(np.sum(chromosome.grid)/len(chromosome.placeHolders))) / len(chromosome.placeHolders)) * 100
                if chromosome.fitness < 0:
                    chromosome.fitness = 0

    def frange(self, start, stop, step):
        steps = []
        while start <= stop:
            steps.append(start)
            start +=step
            
        return steps

    def MakeSensorCombinations(self, start, end, epsilon, sensorType, room):
        a1, b1 = self.makeBoundaries(epsilon, start[0], end[0])
        a2, b2 = self.makeBoundaries(epsilon, start[1], end[1])    
        Xs = self.frange(a1, b1, epsilon)
        Ys = self.frange(a2, b2, epsilon)
        
        points = list(itertools.product(list(itertools.product(Xs, Ys)), [room], [sensorType[0]])) 
        C = itertools.combinations(points, distribution[room][types.index(sensorType)])

        return C

    def PreProcessor(self, df):
        # df['motion sensors'] = df['motion sensors'].apply(ast.literal_eval)
        df['motion sensors'] = df['motion sensors'].apply(lambda s: list(map(int, s)))
        # df['beacon sensors'] = df['beacon sensors'].apply(ast.literal_eval)
        try:
          df['beacon sensors'] = df['beacon sensors'].apply(lambda s: list(map(int, s)))
        except:
          pass

        sensors = set([])

        previous_M = None
        previous_B = None
        output_file = []

        for index, row in df.iterrows():
          T = row['time']
          M = row['motion sensors']
          try:
            B = row['beacon sensors']
          except:
            pass

          Activity = row['activity']
          Activity = Activity.replace(' ', '_')
          MotionSensor_Names = []
          sensorNames = []
          MotionSensor_Message = []
          BeaconSensor_Names = []
          BeaconSensor_Message = []
          

          # time = convertTime(T)
          time = "2020-06-16 " + T + ".00"

          # Motion Sensor
          for i in range(len(M)):
            sensorNames.append(self.Name(i, 'M'))
            if M[i] == 1:
              if (previous_M != None):
                if (previous_M[i] == 0):
                  MotionSensor_Names.append(self.Name(i,'M'))
                  MotionSensor_Message.append('ON')

              else:
                MotionSensor_Names.append(self.Name(i,'M'))
                MotionSensor_Message.append('ON')

            if previous_M != None:
              if M[i] == 0 and previous_M[i] == 1:
                MotionSensor_Names.append(self.Name(i,'M'))
                MotionSensor_Message.append('OFF')

          previous_M = M
          # Beacon Sensor

          try:
            for i in range(len(B)):
              sensorNames.append(self.Name(i, 'B'))
              if B[i] == 1:
                BeaconSensor_Names.append(self.Name(i,'B'))
                BeaconSensor_Message.append('ON')
              if previous_B != None:
                if B[i] == 0 and previous_B[i] == 1: 
                  BeaconSensor_Names.append(self.Name(i,'B'))
                  BeaconSensor_Message.append('OFF')
            previous_B = B

          except:
            pass

          for m in range(len(MotionSensor_Names)):
            output_file.append(time +' '+ MotionSensor_Names[m] + ' ' + MotionSensor_Names[m] + ' ' + MotionSensor_Message[m] + ' ' + Activity)
            
          for s in sensorNames:
              sensors.add(s)

        return output_file, list(sensors)

    #returns the name of the sensor
    def Name(self, number, typeSensor):
        if number < 10:
          return typeSensor + str(0) + str(number)
        else:
          return typeSensor + str(number)

    #converts epoch time to human readable
    def convertTime(self, posix_timestamp):
        tz = pytz.timezone('MST')
        dt = datetime.fromtimestamp(posix_timestamp, tz)
        time = dt.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
        return time

    def BestAnswer(self):
        bestAnswerIndex = 0
        for index, c in enumerate(self.chromosomes):
            if c.fitness > self.chromosomes[bestAnswerIndex].fitness:
                bestAnswerIndex = index

        return self.chromosomes[bestAnswerIndex]

    def AverageAnswer(self):
        return np.sum([c.fitness for c in self.chromosomes]) / len(self.chromosomes)


In [2]:
def PreProcessor(df):
        # df['motion sensors'] = df['motion sensors'].apply(ast.literal_eval)
        df['motion sensors'] = df['motion sensors'].apply(lambda s: list(map(int, s)))
        # df['beacon sensors'] = df['beacon sensors'].apply(ast.literal_eval)
        try:
          df['beacon sensors'] = df['beacon sensors'].apply(lambda s: list(map(int, s)))
        except:
          pass

        sensors = set([])

        previous_M = None
        previous_B = None
        output_file = []

        for index, row in df.iterrows():
          T = row['time']
          M = row['motion sensors']
          try:
            B = row['beacon sensors']
          except:
            pass

          Activity = row['activity']
          Activity = Activity.replace(' ', '_')
          MotionSensor_Names = []
          sensorNames = []
          MotionSensor_Message = []
          BeaconSensor_Names = []
          BeaconSensor_Message = []
          

          # time = convertTime(T)
          time = "2020-06-16 " + T + ".00"

          # Motion Sensor
          for i in range(len(M)):
            sensorNames.append(Name(i, 'M'))
            if M[i] == 1:
              if (previous_M != None):
                if (previous_M[i] == 0):
                  MotionSensor_Names.append(Name(i,'M'))
                  MotionSensor_Message.append('ON')

              else:
                MotionSensor_Names.append(Name(i,'M'))
                MotionSensor_Message.append('ON')

            if previous_M != None:
              if M[i] == 0 and previous_M[i] == 1:
                MotionSensor_Names.append(Name(i,'M'))
                MotionSensor_Message.append('OFF')

          previous_M = M
          # Beacon Sensor

          try:
            for i in range(len(B)):
              sensorNames.append(Name(i, 'B'))
              if B[i] == 1:
                BeaconSensor_Names.append(Name(i,'B'))
                BeaconSensor_Message.append('ON')
              if previous_B != None:
                if B[i] == 0 and previous_B[i] == 1: 
                  BeaconSensor_Names.append(Name(i,'B'))
                  BeaconSensor_Message.append('OFF')
            previous_B = B

          except:
            pass

          for m in range(len(MotionSensor_Names)):
            output_file.append(time +' '+ MotionSensor_Names[m] + ' ' + MotionSensor_Names[m] + ' ' + MotionSensor_Message[m] + ' ' + Activity)
            
          for s in sensorNames:
              sensors.add(s)

        return output_file, list(sensors)

#returns the name of the sensor
def Name(number, typeSensor):
    if number < 10:
      return typeSensor + str(0) + str(number)
    else:
      return typeSensor + str(number)

#converts epoch time to human readable
def convertTime(posix_timestamp):
    tz = pytz.timezone('MST')
    dt = datetime.fromtimestamp(posix_timestamp, tz)
    time = dt.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    return time

def ModelsInitializations(Data_path):
    #----- Space and agent models -----: 
    simworldname = Data_path + '/Configuration Files/simulationWorld2.xml'
    agentTraces = []
    directory = os.fsencode(Data_path)

    print(directory)
    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            agentTraces.append(Data_path + filename)

    # Parsing the space model: 
    space, rooms = pf.ParseWorld(simworldname)

    xs = []
    for i in space:
      for j in i:
        xs.append(j)
    A = list(set(xs))
    A.sort()
    space = [A[-1], A[-2]]

    # User parameters 
    types, sensor_distribution = pf.GetUsersParameters()

    roomsList = []
    for room in sensor_distribution:
        roomsList.append(room)

    return sensor_distribution, types, space, rooms, agentTraces


def Train(Data_path, bestChromosome):
    sensor_distribution, types, space, rooms, agentTraces = ModelsInitializations(Data_path + 'Agent Trace Files/')
    files = []

    # sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
    # Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'

    sys.path.append('../../Codes/Sensor Simulator/')
    Data_path = '../../Codes/Sensor Simulator/'

    all_sensors = set([])

    for agentTrace in agentTraces:
        filecoding = ' '#"_" + str(iteration) + "_c" + str(index + 1) + '(' + self.mode + ')'
        df_ = sim_sis.RunSimulator(space, rooms, agentTrace, bestChromosome.GetSensorConfiguration() , simulateMotionSensors, simulateEstimotes, Plotting, filecoding , Data_path)
        dataFile, sensors = PreProcessor(df_)
        all_sensors.update(sensors)
        #self.D = dataFile
        files.append(dataFile)



    # sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome')
    sys.path.append('../../Codes/CASAS/AL-Smarthome')

    # %cd 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome'

    #import al
    #import imp
    imp.reload(al)
    all_sensors = list(all_sensors)

    cf = al.Train(files, all_sensors)
    
    return cf
    
    
    # chromosome.fitness = (al.leave_one_out(files, all_sensors)[0] - (sum(chromosome.grid) / 100)) * 100 # - (len(chromosome.placeHolders)**(np.sum(chromosome.grid)/len(chromosome.placeHolders))) / len(chromosome.placeHolders)) * 100
    # if chromosome.fitness < 0:
    #     chromosome.fitness = 0


def Test(cf, bestChromosome, Data_path):
    sensor_distribution, types, space, rooms, agentTraces = ModelsInitializations(Data_path + 'Agent Trace Files Test/')
    files = []

    # sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
    # Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'

    sys.path.append('../../Codes/Sensor Simulator/')
    Data_path = '../../Codes/Sensor Simulator/'

    all_sensors = set([])

    for agentTrace in agentTraces:
        filecoding = ' '#"_" + str(iteration) + "_c" + str(index + 1) + '(' + self.mode + ')'
        df_ = sim_sis.RunSimulator(space, rooms, agentTrace, bestChromosome.GetSensorConfiguration() , simulateMotionSensors, simulateEstimotes, Plotting, filecoding , Data_path)
        dataFile, sensors = PreProcessor(df_)
        all_sensors.update(sensors)
        #self.D = dataFile
        files.append(dataFile)

    # sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome')
    sys.path.append('../../Codes/CASAS/AL-Smarthome')

    # %cd 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome'

    #import al
    #import imp
    imp.reload(al)
    all_sensors = list(all_sensors)

    result = al.Test(cf, files, all_sensors)
    
    print(result)
    
    
    
    # chromosome.fitness = (al.leave_one_out(files, all_sensors)[0] - (sum(chromosome.grid) / 100)) * 100 # - (len(chromosome.placeHolders)**(np.sum(chromosome.grid)/len(chromosome.placeHolders))) / len(chromosome.placeHolders)) * 100
    # if chromosome.fitness < 0:
    #     chromosome.fitness = 0


def executeFinalTest(ga):
    import al
    import imp
    simulateMotionSensors, simulateEstimotes, Plotting, iteration = True, False, False, 1
    bestChromosome = ga.chromosomes[0]

    Data_path = '../../Codes/Sensor Simulator/'
    cf = Train(Data_path, bestChromosome)

    print('Training Done!')

    Data_path = '../../Codes/Sensor Simulator/'
    Test(cf, bestChromosome, Data_path)


In [3]:
if __name__ ==  '__main__':
    # from google.colab import drive
    import sys
    # drive.mount('/content/gdrive', force_remount=True)
    
    # Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'
    # sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
    
    Data_path = '../../Codes/Sensor Simulator/'
    sys.path.append('../../Codes/Sensor Simulator/')
    
    from ipywidgets import IntProgress
    from IPython.display import display
    import SIM_SIS_Libraries.SensorsClass
    import SIM_SIS_Libraries.SIM_SIS_Simulator as sim_sis
    import SIM_SIS_Libraries.ParseFunctions as pf
    import itertools
    import numpy as np
    import pandas as pd
    import sys
    import SIM_SIS_Libraries.PreDeploymentEvaluation as pde
    import copy
    from datetime import datetime
    import pytz
    import ast
    import os
    import random

    iteration = 100
    population = 10
    # random_seed = 0.1234
    # random.seed(random_seed)

    # The distance between two nodes in the space grid:
    epsilon = 1 / 8
    
    # initial and max sensor numbers
    initSensorNum = 14
    maxSensorNum = 25

    # radius of the motion sensors
    radius = 1

    # Mutation rate for each item in a chromosome's data (each sensor placeholder)
    mutation_rate = 0.005

    # number of folds in the crossover process
    crossover = 2

    # top % of the sorted chromosomes of each generation that goes to the next generation
    survival_rate = 0.1 

    # top % of the sorted chromosomes of each generation that contributes in breeding children
    reproduction_rate = 0.2

    finalResults = []
    
    for epsilon in [1/4]:
        print('----- Running GA for epsilon = ' + str(epsilon))
        f = IntProgress(min=0, max=iteration) # instantiate the bar
        display(f) # display the bar
        
        ga = GA(population, 
                'expert', 
                Data_path, 
                epsilon, 
                initSensorNum, 
                maxSensorNum, 
                radius, 
                mutation_rate, 
                crossover, 
                survival_rate, 
                reproduction_rate)

        ga.RunFitnessFunction(True, False, False, 1)

        for epoch in range(iteration):
            f.value += 1
            ga.GetNextGeneration()
            ga.RunFitnessFunction(True, False, False, 1)
            ga.Selection()

            print("(epoch %d) ----- The best answer: {%f} with (%d) number of sensors (average fitness is: %f)" 
                  %(epoch + 1, 
                    ga.chromosomes[0].fitness + (sum(ga.chromosomes[0].grid) / 100) * 100, # (len(ga.chromosomes[0].placeHolders)**(np.sum(ga.chromosomes[0].grid)/len(ga.chromosomes[0].placeHolders)) / len(ga.chromosomes[0].placeHolders)) * 100, 
                    np.sum(ga.chromosomes[0].grid),
                    ga.AverageAnswer()))
        
        finalResults.append(executeFinalTest(ga))
        print('--------------------')

            # print("(epoch %d) ----- The best answer so far: {%f} with (%d) number of sensors" %(epoch, ga.BestAnswer().fitness, np.sum(ga.BestAnswer().grid)))

----- Running GA for epsilon = 0.25


IntProgress(value=0)

(epoch 1) ----- The best answer: {69.142158} with (14) number of sensors (average fitness is: 43.306526)
(epoch 2) ----- The best answer: {69.142158} with (14) number of sensors (average fitness is: 41.079012)
(epoch 3) ----- The best answer: {69.142158} with (14) number of sensors (average fitness is: 37.929741)
(epoch 4) ----- The best answer: {69.142158} with (14) number of sensors (average fitness is: 38.430765)
(epoch 5) ----- The best answer: {71.206626} with (15) number of sensors (average fitness is: 41.396715)
(epoch 6) ----- The best answer: {71.206626} with (15) number of sensors (average fitness is: 43.298588)
(epoch 7) ----- The best answer: {71.206626} with (15) number of sensors (average fitness is: 40.498368)
(epoch 8) ----- The best answer: {71.206626} with (15) number of sensors (average fitness is: 48.114496)
(epoch 9) ----- The best answer: {71.206626} with (15) number of sensors (average fitness is: 46.756915)
(epoch 10) ----- The best answer: {76.641255} with (20)

(epoch 79) ----- The best answer: {82.215582} with (25) number of sensors (average fitness is: 42.013127)
(epoch 80) ----- The best answer: {82.215582} with (25) number of sensors (average fitness is: 43.422264)
(epoch 81) ----- The best answer: {82.215582} with (25) number of sensors (average fitness is: 40.422647)
(epoch 82) ----- The best answer: {82.215582} with (25) number of sensors (average fitness is: 42.436816)
(epoch 83) ----- The best answer: {82.215582} with (25) number of sensors (average fitness is: 37.814312)
(epoch 84) ----- The best answer: {82.215582} with (25) number of sensors (average fitness is: 39.720668)
(epoch 85) ----- The best answer: {82.215582} with (25) number of sensors (average fitness is: 42.498838)
(epoch 86) ----- The best answer: {82.215582} with (25) number of sensors (average fitness is: 43.701197)
(epoch 87) ----- The best answer: {82.215582} with (25) number of sensors (average fitness is: 41.171672)
(epoch 88) ----- The best answer: {82.215582} 

KeyboardInterrupt: 

In [4]:
def PreProcessor(df):
        # df['motion sensors'] = df['motion sensors'].apply(ast.literal_eval)
        df['motion sensors'] = df['motion sensors'].apply(lambda s: list(map(int, s)))
        # df['beacon sensors'] = df['beacon sensors'].apply(ast.literal_eval)
        try:
          df['beacon sensors'] = df['beacon sensors'].apply(lambda s: list(map(int, s)))
        except:
          pass

        sensors = set([])

        previous_M = None
        previous_B = None
        output_file = []

        for index, row in df.iterrows():
          T = row['time']
          M = row['motion sensors']
          try:
            B = row['beacon sensors']
          except:
            pass

          Activity = row['activity']
          Activity = Activity.replace(' ', '_')
          MotionSensor_Names = []
          sensorNames = []
          MotionSensor_Message = []
          BeaconSensor_Names = []
          BeaconSensor_Message = []
          

          # time = convertTime(T)
          time = "2020-06-16 " + T + ".00"

          # Motion Sensor
          for i in range(len(M)):
            sensorNames.append(Name(i, 'M'))
            if M[i] == 1:
              if (previous_M != None):
                if (previous_M[i] == 0):
                  MotionSensor_Names.append(Name(i,'M'))
                  MotionSensor_Message.append('ON')

              else:
                MotionSensor_Names.append(Name(i,'M'))
                MotionSensor_Message.append('ON')

            if previous_M != None:
              if M[i] == 0 and previous_M[i] == 1:
                MotionSensor_Names.append(Name(i,'M'))
                MotionSensor_Message.append('OFF')

          previous_M = M
          # Beacon Sensor

          try:
            for i in range(len(B)):
              sensorNames.append(Name(i, 'B'))
              if B[i] == 1:
                BeaconSensor_Names.append(Name(i,'B'))
                BeaconSensor_Message.append('ON')
              if previous_B != None:
                if B[i] == 0 and previous_B[i] == 1: 
                  BeaconSensor_Names.append(Name(i,'B'))
                  BeaconSensor_Message.append('OFF')
            previous_B = B

          except:
            pass

          for m in range(len(MotionSensor_Names)):
            output_file.append(time +' '+ MotionSensor_Names[m] + ' ' + MotionSensor_Names[m] + ' ' + MotionSensor_Message[m] + ' ' + Activity)
            
          for s in sensorNames:
              sensors.add(s)

        return output_file, list(sensors)

#returns the name of the sensor
def Name(number, typeSensor):
    if number < 10:
      return typeSensor + str(0) + str(number)
    else:
      return typeSensor + str(number)

#converts epoch time to human readable
def convertTime(posix_timestamp):
    tz = pytz.timezone('MST')
    dt = datetime.fromtimestamp(posix_timestamp, tz)
    time = dt.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    return time

def ModelsInitializations(Data_path):
    #----- Space and agent models -----: 
    simworldname = Data_path + '/Configuration Files/simulationWorld2.xml'
    agentTraces = []
    directory = os.fsencode(Data_path)

    print(directory)
    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            agentTraces.append(Data_path + filename)

    # Parsing the space model: 
    space, rooms = pf.ParseWorld(simworldname)

    xs = []
    for i in space:
      for j in i:
        xs.append(j)
    A = list(set(xs))
    A.sort()
    space = [A[-1], A[-2]]

    # User parameters 
    types, sensor_distribution = pf.GetUsersParameters()

    roomsList = []
    for room in sensor_distribution:
        roomsList.append(room)

    return sensor_distribution, types, space, rooms, agentTraces


def Train(Data_path, bestChromosome):
    sensor_distribution, types, space, rooms, agentTraces = ModelsInitializations(Data_path + 'Agent Trace Files/')
    files = []

    # sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
    # Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'

    sys.path.append('../../Codes/Sensor Simulator/')
    Data_path = '../../Codes/Sensor Simulator/'

    all_sensors = set([])

    for agentTrace in agentTraces:
        filecoding = ' '#"_" + str(iteration) + "_c" + str(index + 1) + '(' + self.mode + ')'
        df_ = sim_sis.RunSimulator(space, rooms, agentTrace, bestChromosome.GetSensorConfiguration() , simulateMotionSensors, simulateEstimotes, Plotting, filecoding , Data_path)
        dataFile, sensors = PreProcessor(df_)
        all_sensors.update(sensors)
        #self.D = dataFile
        files.append(dataFile)



    # sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome')
    sys.path.append('../../Codes/CASAS/AL-Smarthome')

    # %cd 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome'

    #import al
    #import imp
    imp.reload(al)
    all_sensors = list(all_sensors)

    cf = al.Train(files, all_sensors)
    
    return cf
    
    
    # chromosome.fitness = (al.leave_one_out(files, all_sensors)[0] - (sum(chromosome.grid) / 100)) * 100 # - (len(chromosome.placeHolders)**(np.sum(chromosome.grid)/len(chromosome.placeHolders))) / len(chromosome.placeHolders)) * 100
    # if chromosome.fitness < 0:
    #     chromosome.fitness = 0


def Test(cf, bestChromosome, Data_path):
    sensor_distribution, types, space, rooms, agentTraces = ModelsInitializations(Data_path + 'Agent Trace Files Test/')
    files = []

    # sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
    # Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'

    sys.path.append('../../Codes/Sensor Simulator/')
    Data_path = '../../Codes/Sensor Simulator/'

    all_sensors = set([])

    for agentTrace in agentTraces:
        filecoding = ' '#"_" + str(iteration) + "_c" + str(index + 1) + '(' + self.mode + ')'
        df_ = sim_sis.RunSimulator(space, rooms, agentTrace, bestChromosome.GetSensorConfiguration() , simulateMotionSensors, simulateEstimotes, Plotting, filecoding , Data_path)
        dataFile, sensors = PreProcessor(df_)
        all_sensors.update(sensors)
        #self.D = dataFile
        files.append(dataFile)

    # sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome')
    sys.path.append('../../Codes/CASAS/AL-Smarthome')

    # %cd 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome'

    #import al
    #import imp
    imp.reload(al)
    all_sensors = list(all_sensors)

    result = al.Test(cf, files, all_sensors)
    
    print(result)
    
    
    
    # chromosome.fitness = (al.leave_one_out(files, all_sensors)[0] - (sum(chromosome.grid) / 100)) * 100 # - (len(chromosome.placeHolders)**(np.sum(chromosome.grid)/len(chromosome.placeHolders))) / len(chromosome.placeHolders)) * 100
    # if chromosome.fitness < 0:
    #     chromosome.fitness = 0



import al
import imp
simulateMotionSensors, simulateEstimotes, Plotting, iteration = True, False, False, 1
bestChromosome = ga.chromosomes[0]

Data_path = '../../Codes/Sensor Simulator/'
cf = Train(Data_path, bestChromosome)

print('Training Done!')

Data_path = '../../Codes/Sensor Simulator/'
Test(cf, bestChromosome, Data_path)


b'../../Codes/Sensor Simulator/Agent Trace Files/'
Training Done!
b'../../Codes/Sensor Simulator/Agent Trace Files Test/'
(0.7104359685803503, 0.010597480206020082)


In [ ]:
# SENSORS VISUALIZATION

x = []
y = []
for i, g in enumerate(ga.chromosomes[0].grid):
    # print(g)
    if g == 1:
        x.append(ga.chromosomes[0].placeHolders[i][0])
        y.append(ga.chromosomes[0].placeHolders[i][1])

import matplotlib.pyplot as plt
plt.figure(figsize=(6.6, 10))
plt.scatter(y, x)